In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import DBSCAN
from sklearn.metrics import silhouette_score, calinski_harabasz_score, davies_bouldin_score
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
df = pd.read_csv('../TWO_CENTURIES_ANALYSIS_CLEAN_WITH_PACE.csv')

In [ ]:
# Variables numéricas para aplicar agrupamiento
variables = [
    'Distance_km',
    'Avg_speed_kmh',
    'Athlete age',
    'Time_hours',
    'Speed_calc_kmh',
    'Speed_diff',
    'Pace_min_per_km'
]

# Semillas para generar diferentes muestras del dataset
semillas = [42, 99, 123]
resultados = []

# Esta función toma un subconjunto del DataFrame, selecciona las columnas relevantes y normaliza los datos
def preparar_datos(df_sample):
    df_sel = df_sample[variables].dropna()
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(df_sel)
    return X_scaled, df_sel

# Esta función aplica el algoritmo DBSCAN sobre los datos escalados
# y calcula las métricas internas (solo si hay más de un cluster encontrado)
def aplicar_dbscan(X, eps=0.6, min_samples=10):
    modelo = DBSCAN(eps=eps, min_samples=min_samples)
    etiquetas = modelo.fit_predict(X)
    n_clusters = len(set(etiquetas)) - (1 if -1 in etiquetas else 0)

    if n_clusters <= 1:
        # No se pueden calcular métricas si solo hay un cluster (o ninguno)
        return etiquetas, n_clusters, None, None, None

    sil = silhouette_score(X, etiquetas)
    calinski = calinski_harabasz_score(X, etiquetas)
    davies = davies_bouldin_score(X, etiquetas)
    return etiquetas, n_clusters, sil, calinski, davies

# Esta función usa PCA para reducir los datos a 2D y graficar los clusters
def graficar_clusters(X, etiquetas, titulo):
    pca = PCA(n_components=2)
    X_pca = pca.fit_transform(X)
    plt.figure(figsize=(6, 5))
    sns.scatterplot(x=X_pca[:, 0], y=X_pca[:, 1], hue=etiquetas, palette='tab10', s=10, legend='full')
    plt.title(titulo)
    plt.xlabel("PCA 1")
    plt.ylabel("PCA 2")
    plt.legend(title="Cluster", loc="best", bbox_to_anchor=(1, 1))
    plt.tight_layout()
    plt.show()

# Ejecutar DBSCAN sobre 3 muestras distintas del dataset
for i, seed in enumerate(semillas):
    print(f"Procesando muestra {i+1} con semilla {seed}...")
    df_muestra = df.sample(n=50000, random_state=seed)
    X_scaled, df_sel = preparar_datos(df_muestra)
    etiquetas, n_clusters, sil, calinski, davies = aplicar_dbscan(X_scaled, eps=0.6, min_samples=10)

    resultados.append({
        'Muestra': i+1,
        'Clusters detectados': n_clusters,
        'Silhouette': sil,
        'Calinski-Harabasz': calinski,
        'Davies-Bouldin': davies
    })

    graficar_clusters(X_scaled, etiquetas, f"DBSCAN - Muestra {i+1} (clusters: {n_clusters})")

# Mostrar tabla de resultados
pd.DataFrame(resultados)